# 4 处理 DNA.csv 文件

In [2]:
import pandas as pd
import numpy as np

In [3]:
# 读取文件，需要分批读取，由于文件过大 30G+
dna_file = []
dnas = pd.read_csv('../DNA.csv', index_col=0, iterator=True, chunksize=1000)
for dna in dnas:
    dna_file.append(dna)

In [20]:
# 过滤 NA/0 值超过 70% 的行和列

len_dna_file = len(dna_file)
len_col = len(dna_file[0].columns)

for l in range(len_dna_file):
    drop_index = [ ]
    for i in range(len(dna_file[l])):
        if(dna_file[l].iloc[i, :].isna().sum() >= len_col*0.7):
            drop_index.append(dna_file[l].index[i])
    dna_file[l].drop(drop_index, axis=0, inplace=True)
    print(str(l)+' -th ', ': ', drop_index)

0 -th  :  []
1 -th  :  []
2 -th  :  []
3 -th  :  []
4 -th  :  []
5 -th  :  []
6 -th  :  []
7 -th  :  []
8 -th  :  []
9 -th  :  []
10 -th  :  []
11 -th  :  []
12 -th  :  []
13 -th  :  []
14 -th  :  []
15 -th  :  []
16 -th  :  []
17 -th  :  []
18 -th  :  []
19 -th  :  []
20 -th  :  []
21 -th  :  []
22 -th  :  []
23 -th  :  []
24 -th  :  []
25 -th  :  []
26 -th  :  []
27 -th  :  []
28 -th  :  []
29 -th  :  []
30 -th  :  []
31 -th  :  []
32 -th  :  []
33 -th  :  []
34 -th  :  []
35 -th  :  []
36 -th  :  []
37 -th  :  []
38 -th  :  []
39 -th  :  []
40 -th  :  []
41 -th  :  []
42 -th  :  []
43 -th  :  []
44 -th  :  []
45 -th  :  []
46 -th  :  []
47 -th  :  []
48 -th  :  []
49 -th  :  []
50 -th  :  []
51 -th  :  []
52 -th  :  []
53 -th  :  []
54 -th  :  []
55 -th  :  []
56 -th  :  []
57 -th  :  []
58 -th  :  []
59 -th  :  []
60 -th  :  []
61 -th  :  []
62 -th  :  []
63 -th  :  []
64 -th  :  []
65 -th  :  []
66 -th  :  []
67 -th  :  []
68 -th  :  []
69 -th  :  []
70 -th  :  []
71 -th  :  []
72

In [22]:
len_index = len(dna_file[0].index) * (len_dna_file-1) + len(dna_file[-1].index)
print("基因数量： ", len_index)
drop_col = [ ]
for i in range(len_col):
    na_sum = 0
    for l in range(len_dna_file):
        na_sum += dna_file[l].iloc[:, i].isna().sum()
    if(na_sum >= len_index*0.7):
        drop_col.append(dna_file[0].index[i])
print("将被过滤的样本： ", drop_col)

基因数量：  396065
将被过滤的样本：  []


In [23]:
# 将剩下的 NA 值以该行的均值替代
for l in range(len_dna_file):
    for i in range(len(dna_file[l].index)):
        dna_file[l].iloc[i, :].fillna(value=dna_file[l].iloc[i, :].mean(), axis=0, inplace=True)

In [24]:
# 确认是否还存在 NA 值
test_na_Sum = 0

for l in range(len_dna_file):
    test_na_Sum += sum(dna_file[l].isna().sum())

print("此时NA值的数量为: ", test_na_Sum)

此时NA值的数量为:  0


In [27]:
# 保存文件
dna_file[0].to_csv('DNA.csv')

In [28]:
for l in range(1, len_dna_file):
    dna_file[l].to_csv('DNA.csv', header=False, mode='a')

# 5 找出三个组学数据中互相包含的样本

In [30]:
# 获取每个组学所包含的样本数

dna_samples = dna_file[0].columns.tolist()
rna_samples = pd.read_csv('RNA.csv', index_col=0).columns.tolist()
rppa_samples = pd.read_csv('RPPA.csv', index_col=0).columns.tolist()
print("DNA 样本数： ", len(dna_samples), '\n', "RNA 样本数：", len(rna_samples), '\n', "RPPA 样本数：", len(rppa_samples))

DNA 样本数：  9664 
 RNA 样本数： 11069 
 RPPA 样本数： 7754


In [31]:
# 过滤各个组学互不包含的的样本，留下相同的样本
co_samples = [sample for sample in rna_samples if sample in dna_samples and sample in rppa_samples]
print("相互包含的样本数为: {}".format(len(co_samples)))

相互包含的样本数为: 6133


In [32]:
tumor_id = pd.read_csv('../data/common_samples.csv', index_col=0)

In [36]:
for sample in co_samples: assert sample in tumor_id['Samples'].tolist()

In [39]:
with open('samples.txt', 'w', encoding='utf-8') as f:
    for i in range(len(tumor_id)):
        temp = tumor_id[['Samples', 'Tumor_id']].iloc[i, :].tolist()
        f.write(temp[0]+' '+str(temp[1])+'\n')

In [40]:
# 处理 **.csv文件，只保留相互包含的样本
rna_file = pd.read_csv('RNA.csv', index_col=0)
rppa_file = pd.read_csv('RPPA.csv', index_col=0)

In [47]:
rna_file = rna_file[co_samples]
rna_file.to_csv('rna.csv')
rppa_file = rppa_file[co_samples]
rppa_file.to_csv('rppa.csv')

In [48]:
temp = dna_file[0][co_samples]
temp.to_csv('dna.csv')
for l in range(1, len_dna_file):
    temp = dna_file[l][co_samples]
    temp.to_csv('dna.csv', mode='a', header=False)

# 6 拆分数据集

In [64]:
samples_dict = {}
for i in range(33):
    samples_dict[i] = tumor_id.loc[tumor_id['Tumor_id'] == i]['Samples'].tolist()

In [69]:
# 获取各类肿瘤 样本数量
classer_num = []
for i in samples_dict.values():
    classer_num.append(len(i))

In [86]:
# 划分数据集
train_ref_num, va_ref_num, test_ref_num = [], [], []

train_samples = {}    # 40%
validation_samples = {}  # 30%
test_samples = {}    # 30%
for idx, i in enumerate(classer_num):
    tr_n = int(i*0.4)
    train_ref_num.append(tr_n)
    v_n = int(i*0.3)
    va_ref_num.append(v_n)
    te_n = i - tr_n - v_n
    test_ref_num.append(te_n)
    train_samples[idx] = samples_dict[idx][0:tr_n]
    validation_samples[idx] = samples_dict[idx][tr_n: tr_n+v_n]
    test_samples[idx] = samples_dict[idx][tr_n+v_n: ]
    print("肿瘤名： {}, 标签： {}, 训练集数量：{}，验证集数量：{}， 测试集数量：{}"
    .format(tumor_id[(tumor_id['Tumor_id'] == idx)].index.tolist()[0], idx, tr_n, v_n, te_n))

肿瘤名： Testicular Germ Cell Tumors, 标签： 0, 训练集数量：48，验证集数量：36， 测试集数量：38
肿瘤名： Head and Neck squamous cell carcinoma, 标签： 1, 训练集数量：135，验证集数量：101， 测试集数量：102
肿瘤名： Breast invasive carcinoma, 标签： 2, 训练集数量：250，验证集数量：187， 测试集数量：189
肿瘤名： Bladder Urothelial Carcinoma, 标签： 3, 训练集数量：167，验证集数量：125， 测试集数量：127
肿瘤名： Lymphoid Neoplasm Diffuse Large B-cell Lymphoma, 标签： 4, 训练集数量：13，验证集数量：9， 测试集数量：11
肿瘤名： Cervical squamous cell carcinoma and endocervical adenocarcinoma, 标签： 5, 训练集数量：67，验证集数量：50， 测试集数量：51
肿瘤名： Liver hepatocellular carcinoma, 标签： 6, 训练集数量：71，验证集数量：53， 测试集数量：54
肿瘤名： Rectum adenocarcinoma, 标签： 7, 训练集数量：29，验证集数量：21， 测试集数量：23
肿瘤名： Kidney renal clear cell carcinoma, 标签： 8, 训练集数量：129，验证集数量：96， 测试集数量：98
肿瘤名： Stomach adenocarcinoma, 标签： 9, 训练集数量：117，验证集数量：87， 测试集数量：89
肿瘤名： Colon adenocarcinoma, 标签： 10, 训练集数量：95，验证集数量：71， 测试集数量：73
肿瘤名： Sarcoma, 标签： 11, 训练集数量：87，验证集数量：65， 测试集数量：67
肿瘤名： Esophageal carcinoma, 标签： 12, 训练集数量：49，验证集数量：37， 测试集数量：38
肿瘤名： Pheochromocytoma and Paraganglioma, 标签： 13, 训练集数量：32，验证

In [88]:
# 存储划分的数据（未进行过采样处理）
with open('origin_classify_samples.txt', 'w', encoding='utf-8') as f:
    f.write("Train samples data: \n")
    for i in range(len(train_samples)):
        for j in train_samples[i]:
            f.write(j+' '+str(i)+'\n')
    f.write("Validation samples data: \n")
    for i in range(len(validation_samples)):
        for j in validation_samples[i]:
            f.write(j+' '+str(i)+'\n')
    f.write("Test samples data: \n")
    for i in range(len(test_samples)):
        for j in test_samples[i]:
            f.write(j+' '+str(i)+'\n')            

In [77]:
# 对少数样本过采样， 减小数据不平衡带来的影响
from random import choice

In [90]:
train_max_num, val_max_num, test_max_num = np.max(train_ref_num), np.max(va_ref_num), np.max(test_ref_num)
for i in range(len(samples_dict)):
    
    if(train_ref_num[i] < train_max_num):
        temp = len(train_samples[i])
        for j in range(train_max_num - temp):
            train_samples[i].append(choice(train_samples[i]))

    if(va_ref_num[i] < val_max_num):
        temp = len(validation_samples[i])
        for j in range(val_max_num - temp):
            validation_samples[i].append(choice(validation_samples[i]))  

    if(test_ref_num[i] < test_max_num):
        temp = len(test_samples[i])
        for j in range(test_max_num - temp):
            test_samples[i].append(choice(test_samples[i]))              

In [92]:
# 检验过采样是否成功
for i in range(len(train_samples)): assert len(train_samples[i]) == train_max_num
for i in range(len(validation_samples)): assert len(validation_samples[i]) == val_max_num
for i in range(len(test_samples)): assert len(test_samples[i]) == test_max_num

In [93]:
# 保存采样之后的数据集
with open('../my_dataset/train.txt', 'w', encoding='utf-8') as train_f, \
     open('../my_dataset/validation.txt', 'w', encoding='utf-8') as validation_f, \
     open('../my_dataset/test.txt', 'w', encoding='utf-8') as test_f:
    for i in range(len(train_samples)):
        for j in train_samples[i]:
            train_f.write(j+' '+str(i)+'\n')
    for i in range(len(validation_samples)):
        for j in validation_samples[i]:
            validation_f.write(j+' '+str(i)+'\n')   
    for i in range(len(test_samples)):
        for j in test_samples[i]:
            test_f.write(j+' '+str(i)+'\n')   

# 7 制作数据集

In [126]:
train_samples_list = [s for i in range(len(train_samples)) for s in train_samples[i]]
validation_samples_list = [s for i in range(len(validation_samples)) for s in validation_samples[i]]
test_samples_list = [s for i in range(len(test_samples)) for s in test_samples[i]]

for sample in co_samples:
    temp_dna = dna_file[0][sample].tolist()
    for l in range(1, len(dna_file)):
        temp_dna.append(dna_file[l][sample].tolist())
    temp_dna = str(temp_dna).replace('[', '').replace(']', '')
    temp_rna = str(rna_file[sample].tolist()).replace('[', '').replace(']', '')
    temp_rppa = str(rppa_file[sample].tolist()).replace('[', '').replace(']', '')    
    
    if sample in train_samples_list:
        with open('../data/train/dna/{}.txt'.format(sample), 'w', encoding='utf-8') as dna_f, \
             open('../data/train/rna/{}.txt'.format(sample), 'w', encoding='utf-8') as rna_f, \
             open('../data/train/rppa/{}.txt'.format(sample), 'w', encoding='utf-8') as rppa_f:
             dna_f.write(temp_dna)
             rna_f.write(temp_rna)
             rppa_f.write(temp_rppa)
    elif sample in validation_samples_list:
        with open('../data/validation/dna/{}.txt'.format(sample), 'w', encoding='utf-8') as dna_f, \
             open('../data/validation/rna/{}.txt'.format(sample), 'w', encoding='utf-8') as rna_f, \
             open('../data/validation/rppa/{}.txt'.format(sample), 'w', encoding='utf-8') as rppa_f:
             dna_f.write(temp_dna)
             rna_f.write(temp_rna)
             rppa_f.write(temp_rppa)       
    elif sample in test_samples_list:
        with open('../data/test/dna/{}.txt'.format(sample), 'w', encoding='utf-8') as dna_f, \
             open('../data/test/rna/{}.txt'.format(sample), 'w', encoding='utf-8') as rna_f, \
             open('../data/test/rppa/{}.txt'.format(sample), 'w', encoding='utf-8') as rppa_f:
             dna_f.write(temp_dna)
             rna_f.write(temp_rna)
             rppa_f.write(temp_rppa)            

# *附：清除DNA中未匹配染色体的探针

In [3]:
# 读取文件，需要分批读取，由于文件过大 30G+
dna_file = []
dnas = pd.read_csv('DNA.csv', index_col=0, iterator=True, chunksize=1000)
for dna in dnas:
    dna_file.append(dna)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
hg38 = pd.read_csv('illuminaMethyl450_hg38_GDC')